# Metaflow Decorators

Using Metaflow requires the use of decorators. In Python code, a decorator is a function that takes another function and extends its behavior without the need to modify it directly. 

In metaflow, there is  function- or step-level decorators AND there is also class- or flow-level decorators.

## Flow Decorators

https://docs.metaflow.org/api/flow-decorators

These decorators are set at the flow level, i.e. above the class definition. In contrast to step decorators, they can't be attached on the command line using `--with`:

```python
@conda_base(libraries={'pandas': '1.0.0'})
@schedule(weekly=True)
@project(name='myproject')
class MyFlow(FlowSpec):
    ...
```

* `@conda_base` - set libraries used by all steps.
* `@project` - create isolated namespaces around flows.
* `@schedule` - define when to run flows in production.

## Step Decorators

https://docs.metaflow.org/api/step-decorators

These decorators are set at the step level, i.e. above the `@step` decorator.

Notably, you can attach any of these decorators to the flow on the command line as well without changing the code using the `--with` option. For instance,

```
python myflow.py run --with retry --with batch:cpu=4
```

attaches a `@retry` and `@batch(cpu=4)` decorators to all steps of the flow.

* `@environment` - define environment variables for containers.
* `@batch` - run on AWS Batch.
* `@card` - enable reporting.
* `@catch` - catch errors.
* `@conda` - define libraries.
* `@kubernetes` - run on Kubernetes.
* `@resources` - request resources.
* `@retry` - retry errors.
* `@step` - define steps.
* `@timeout` - avoid getting stuck.

# An example

Let's create a `.py` file named `decorator_flow.py` and paste the Flow code below:

In [ ]:
from metaflow import FlowSpec, step, card


class DecoratorFlow(FlowSpec):
    
    @step
    def start(self):
        self.next(self.my_decorated_func)
        
    @card
    @step
    def my_decorated_func(self):
        self.data = [1, 2, 3]
        self.next(self.end)
    
    @step
    def end(self):
        print("Flow is done!")

if __name__ == "__main__":
    DecoratorFlow()

Let's run the flow with the command:

```bash
python3.10 decorator_flow.py run
```


```
Arat-MacBook-Pro-2:metaflow_example mustafamuratarat$ python3.10 decorator_flow.py run
Metaflow 2.8.0 executing DecoratorFlow for user:mustafamuratarat
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2023-03-12 21:09:32.123 Workflow starting (run-id 1678644572114155):
2023-03-12 21:09:32.130 [1678644572114155/start/1 (pid 38424)] Task is starting.
2023-03-12 21:09:32.730 [1678644572114155/start/1 (pid 38424)] Task finished successfully.
2023-03-12 21:09:32.740 [1678644572114155/my_decorated_func/2 (pid 38427)] Task is starting.
2023-03-12 21:09:33.899 [1678644572114155/my_decorated_func/2 (pid 38427)] Task finished successfully.
2023-03-12 21:09:33.908 [1678644572114155/end/3 (pid 38435)] Task is starting.
2023-03-12 21:09:34.392 [1678644572114155/end/3 (pid 38435)] Flow is done!
2023-03-12 21:09:34.470 [1678644572114155/end/3 (pid 38435)] Task finished successfully.
2023-03-12 21:09:34.471 Done!
```

Now that we have run `@card` for the `my_decorated_func` step, we can use the following command to visualize our flow:

```
Arat-MacBook-Pro-2:metaflow_example mustafamuratarat$ python3.10 decorator_flow.py card view my_decorated_func
Metaflow 2.8.0 executing DecoratorFlow for user:mustafamuratarat
Resolving card: DecoratorFlow/1678644572114155/my_decorated_func/2
```

You should now see a browser tab open where you can inspect flow results.

![](https://github.com/mmuratarat/metaflow_tutorial/blob/main/images/Screenshot%202023-03-12%20at%209.12.53%20PM.png?raw=true)
![](https://github.com/mmuratarat/metaflow_tutorial/blob/main/images/Screenshot%202023-03-12%20at%209.12.48%20PM.png?raw=true)